In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
from unidecode import unidecode
from selenium.webdriver.common.keys import Keys
import pandas as pd
from collections import defaultdict
import json
from datetime import datetime

driver = webdriver.Chrome('/Users/ik/Data/webdrivers/chromedriver')
WAIT_TIME = 30

In [6]:
emails = pd.read_csv("/Users/ik/Data/email-domains/verified_email_domains.txt", sep=":", header=None, names="domain status".split())

In [ ]:
ok_emails = sorted(list(set(emails.loc[emails.status=='ok']["domain"].tolist())))
print("total ok emails: {}".format(len(ok_emails)))

total ok emails: 360181


In [ ]:
comp_info = []  # list of all company information pieces

for i, company_domain in enumerate(ok_emails[:10], 1):
    
    print("#{}: searching for {}...".format(i, company_domain))
    
    driver.get("https://www.google.com.au/")
    SUBJ_WORD = " ".join(["linkedin", company_domain])
    WebDriverWait(driver, WAIT_TIME).until(EC.presence_of_element_located((By.XPATH, "//input[@id='lst-ib']"))).send_keys(SUBJ_WORD)
    driver.find_element_by_xpath("//input[@type='submit']").click()

    h3 = driver.find_element_by_xpath("//div[@class='s']")
    linkedin_url = h3.find_element_by_xpath(".//cite[@class='_Rm']").text
    
    if "au.linkedin.com" in linkedin_url:
        
        driver.get(linkedin_url)
        
        try:
            WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.CLASS_NAME, "public-follow")))
        except:
            print("no follow button, skipping..")
            continue
            
        try:
            cname = driver.find_element_by_xpath("//h1[@class='name']")
        except:
            # we don't care about companies without names
            continue
        
        try:
            indus = driver.find_element_by_xpath("//p[@class='industry']")
        except:
            indus = None
            
        try:
            csize = driver.find_element_by_xpath("//p[@class='company-size']")
        except:
            size = None
            
        try:
            cwebs = driver.find_element_by_xpath("//li[@class='website']/p")
        except:
            cwebs = None
        
        try:
            ctype = driver.find_element_by_xpath("//li[@class='type']/p")
        except:
            ctype = None
        
        try:
            cadr = driver.find_element_by_xpath("//p[@class='adr']")
        except:
            cadr = None
            
        comp_info.append({k: v.text.lower().strip() if v else None for k, v in {"name": cname, "industry": indus, "size": csize,
                                "website": cwebs, "type": ctype, "address": cadr}.items()})
    
        print(comp_info)
        
    else:
        print("can\'t find this company on linkedin, moving on...".format(company_domain.upper()))     

json.dump(comp_info, open('linkedin-company-info.json', 'w'))
driver.quit()

#1: searching for 0014.com.au...
